# Analizador Lexico (Con más de una función)

### Analisis Léxico

In [ ]:
import re
import json
import os
import subprocess
import platform

In [ ]:
# === Analisis léxico ===
# Definir los patrones

token_patron = {
    "KEYWORD": r"\b(if|else|while|return|int|float|void|for|print)\b",
    "IDENTIFIER": r"\b[a-zA-Z_][a-zA-Z0-9_]*\b",
    "NUMBER": r"\b\d+(\.\d+)?\b",
    "OPERATOR": r"(==|!=|<=|>=|&&|\|\||\+\+|--|[+\-*/=<>!&])",
    "STRING": r'"([^"]*)"',
    "DELIMITER": r"[(),;{}]",
    "WHITESPACE": r"\s+",
}

def identificar(texto):
    # Unir todos los patrones en un único patrón utilizando grupos nombrados
    patron_general = "|".join(f"(?P<{token}>{patron})" for token, patron in token_patron.items())
    patron_regex = re.compile(patron_general)
    tokens_encontrados = []
    for match in patron_regex.finditer(texto):
        for token, valor in match.groupdict().items():
            if valor is not None and token != "WHITESPACE":
                tokens_encontrados.append((token, valor))
    return tokens_encontrados

### Parsear

In [ ]:
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = 0
        
    def obtener_token_actual(self):
        return self.tokens[self.pos] if self.pos < len(self.tokens) else None

    def coincidir(self, tipo_esperado):
        token_actual = self.obtener_token_actual()
        if token_actual and token_actual[0] == tipo_esperado:
            self.pos += 1
            return token_actual
        else:
            raise SyntaxError(f'Error sintactico: se esperaba {tipo_esperado}, pero se encontro: {token_actual}')

    def parsear(self):
        # Ahora parsear retorna un nodo programa que contiene todas las funciones
        funciones = []
        
        while self.pos < len(self.tokens):
            funciones.append(self.funcion())
            
        programa = NodoPrograma(funciones)
        
        # Verificar que exista la función main
        if not programa.tiene_main():
            print("Advertencia: No se encontró la función main() en el programa")
            
        return programa
  
    def funcion(self):
        tipo_token = self.coincidir('KEYWORD')  # Tipo de retorno (e). int()
        tipo = tipo_token[1]
        nombre_token = self.coincidir('IDENTIFIER')  # Nombre de la funcion
        nombre = nombre_token[1]
        
        self.coincidir('DELIMITER')  # Se espera un parentesis
        parametros = self.parametros()
        self.coincidir('DELIMITER')  # Se espera un parentesis de cierre ")"
        self.coincidir('DELIMITER')  # Se espera una llave de apertura "{"
        
        cuerpo = self.cuerpo()  # Se espera el cuerpo de la funcion
        
        self.coincidir('DELIMITER')  # Se espera una llave de cierre "}"
        
        return NodoFuncion(tipo, nombre, parametros, cuerpo)
    
    def parametros(self):
        # Reglas para parametros: int IDENTIFIER (,int IDENTIFIER)*
        parametros = []
        
        # Verificar si hay parámetros (puede ser una función sin parámetros)
        if self.obtener_token_actual() and self.obtener_token_actual()[1] != ')':
            tipo_token = self.coincidir('KEYWORD')  # Tipo del parametro
            tipo = tipo_token[1]
            nombre_token = self.coincidir('IDENTIFIER')  # Nombre del parametro
            nombre = nombre_token[1]
            
            parametros.append(NodoParametro(tipo, nombre))
            
            while self.obtener_token_actual() and self.obtener_token_actual()[1] == ',':
                self.coincidir('DELIMITER')  # Se espera una coma ","
                tipo_token = self.coincidir('KEYWORD')  # Tipo del parametro (int)
                tipo = tipo_token[1]
                nombre_token = self.coincidir('IDENTIFIER')  # Nombre del parametro
                nombre = nombre_token[1]
                
                parametros.append(NodoParametro(tipo, nombre))
            
        return parametros

    def cuerpo(self):
        instrucciones = []
        
        while self.obtener_token_actual() and self.obtener_token_actual()[1] != '}':
            if self.obtener_token_actual()[1] == 'return':
                instrucciones.append(self.declaracion())
            elif self.obtener_token_actual()[1] == 'while':
                instrucciones.append(self.ciclo_while())
            elif self.obtener_token_actual()[1] == 'if':
                instrucciones.append(self.condicional_if())
            elif self.obtener_token_actual()[1] == 'else':
                instrucciones.append(self.condicional_else())
            elif self.obtener_token_actual()[1] == 'for':
                instrucciones.append(self.ciclo_for())
            elif self.obtener_token_actual()[1] == 'print':
                instrucciones.append(self.imprimir())
            elif self.obtener_token_actual()[0] == 'IDENTIFIER':
                # Verificar si es una llamada a función o un incremento
                siguiente = self.tokens[self.pos + 1] if self.pos + 1 < len(self.tokens) else None
                if siguiente and siguiente[1] == '(':
                    instrucciones.append(self.llamada_funcion())
                    self.coincidir('DELIMITER')  # Punto y coma ";"
                elif siguiente and siguiente[0] == 'OPERATOR' and siguiente[1] in ('++', '--'):
                    instruccion = self.incremento()
                    instrucciones.append(instruccion)
                    self.coincidir('DELIMITER')  # Punto y coma ";"
                else:
                    # Es una asignación a una variable existente
                    nombre_token = self.coincidir('IDENTIFIER')
                    nombre = nombre_token[1]
                    self.coincidir('OPERATOR')  # =
                    expresion = self.expresion()
                    self.coincidir('DELIMITER')  # ;
                    instrucciones.append(NodoAsignacion(None, nombre, expresion))
            elif self.obtener_token_actual()[0] == 'KEYWORD':
                # Es una declaración de variable con posible asignación
                instrucciones.append(self.asignacion())
            else:
                raise SyntaxError(f"Error sintáctico: instrucción inesperada {self.obtener_token_actual()}")
                
        return instrucciones

    def llamada_funcion(self):
        nombre_token = self.coincidir('IDENTIFIER')  # Nombre de la función
        nombre = nombre_token[1]
        
        self.coincidir('DELIMITER')  # (
        
        argumentos = []
        if self.obtener_token_actual() and self.obtener_token_actual()[1] != ')':
            argumentos.append(self.expresion())
            
            while self.obtener_token_actual() and self.obtener_token_actual()[1] == ',':
                self.coincidir('DELIMITER')  # ,
                argumentos.append(self.expresion())
                
        self.coincidir('DELIMITER')  # )
        
        return NodoLlamadaFuncion(nombre, argumentos)

    def asignacion(self):
        tipo_token = self.coincidir('KEYWORD')  # Tipo de variable (ej. int)
        tipo = tipo_token[1]
        nombre_token = self.coincidir('IDENTIFIER')  # Nombre de la variable
        nombre = nombre_token[1]
        self.coincidir('OPERATOR')  # Operador de asignación "="
        expresion = self.expresion()  
        self.coincidir('DELIMITER')  # Punto y coma ";"
        
        return NodoAsignacion(tipo, nombre, expresion)

    def declaracion(self):
        self.coincidir('KEYWORD')  # return
        expresion = self.expresion()
        self.coincidir('DELIMITER')  # Punto y coma ";"
        
        return NodoRetorno(expresion)

    def expresion(self):
        izquierda = self.termino()
        
        while self.obtener_token_actual() and self.obtener_token_actual()[1] in ('+', '-', '==', '!=', '<', '>', '<=', '>=', '&&', '||'):
            operador_token = self.coincidir('OPERATOR')
            operador = operador_token[1]
            derecha = self.termino()
            izquierda = NodoOperacion(izquierda, operador, derecha)
            
        return izquierda

    def termino(self):
        izquierda = self.factor()
        
        while self.obtener_token_actual() and self.obtener_token_actual()[1] in ('*', '/'):
            operador_token = self.coincidir('OPERATOR')
            operador = operador_token[1]
            derecha = self.factor()
            izquierda = NodoOperacion(izquierda, operador, derecha)
            
        return izquierda

    def factor(self):
        token_actual = self.obtener_token_actual()
        
        if token_actual[0] == 'NUMBER':
            token = self.coincidir('NUMBER')
            return NodoNumero(token[1])
        elif token_actual[0] == 'IDENTIFIER':
            # Verificar si es una llamada a función o una variable
            siguiente = self.tokens[self.pos + 1] if self.pos + 1 < len(self.tokens) else None
            if siguiente and siguiente[1] == '(':
                return self.llamada_funcion()
            else:
                token = self.coincidir('IDENTIFIER')
                return NodoIdentificador(token[1])
        elif token_actual[1] == '(':
            self.coincidir('DELIMITER')  # "("
            expresion = self.expresion()
            self.coincidir('DELIMITER')  # ")"
            return expresion
        else:
            raise SyntaxError(f'Error sintáctico en factor: token inesperado {token_actual}')

    def incremento(self):
        nombre_token = self.coincidir('IDENTIFIER')  # Nombre de la variable
        nombre = nombre_token[1]
        
        operador_token = self.coincidir('OPERATOR')  # Coincide con ++ o --
        operador = operador_token[1]
        
        return NodoIncremento(nombre, operador)
      
    def instruccion_unica(self):
        token_actual = self.obtener_token_actual()
        if not token_actual:
            raise SyntaxError("Error sintáctico: se esperaba una instrucción")

        if token_actual[0] == "KEYWORD":
            if token_actual[1] == "return":
                return self.declaracion()
            elif token_actual[1] == "while":
                return self.ciclo_while()
            elif token_actual[1] == "if":
                return self.condicional_if()
            elif token_actual[1] == "else":
                return self.condicional_else()
            elif token_actual[1] == "for":
                return self.ciclo_for()
            elif token_actual[1] == "print":
                return self.imprimir()
            else:
                return self.asignacion()
        elif token_actual[0] == "IDENTIFIER":
            # Verificar si es una llamada a función
            siguiente = self.tokens[self.pos + 1] if self.pos + 1 < len(self.tokens) else None
            if siguiente and siguiente[1] == '(':
                instruccion = self.llamada_funcion()
                self.coincidir('DELIMITER')  # ;
                return instruccion
            else:
                return self.asignacion()
        else:
            raise SyntaxError(f"Error sintáctico: instrucción inesperada {token_actual}")

    def ciclo_while(self):
        self.coincidir('KEYWORD')  # while
        self.coincidir('DELIMITER')  # "("
        condicion = self.expresion()  # Condición del ciclo
        self.coincidir('DELIMITER')  # ")"

        # Verificar si el siguiente token es una llave de apertura "{"
        cuerpo = []
        if self.obtener_token_actual() and self.obtener_token_actual()[1] == "{":
            self.coincidir('DELIMITER')  # "{"
            cuerpo = self.cuerpo()  # Cuerpo del while
            self.coincidir('DELIMITER')  # "}"
        else:
            # Si no hay llaves, solo ejecutamos una única instrucción
            cuerpo.append(self.instruccion_unica())
            
        return NodoWhile(condicion, cuerpo)

    def ciclo_for(self):
        self.coincidir('KEYWORD')  # for
        self.coincidir('DELIMITER')  # "("
        
        # Inicialización
        if self.obtener_token_actual()[0] == 'KEYWORD':
            inicializacion = self.asignacion()
        else:
            nombre_token = self.coincidir('IDENTIFIER')
            nombre = nombre_token[1]
            self.coincidir('OPERATOR')  # =
            expresion = self.expresion()
            self.coincidir('DELIMITER')  # ";"
            inicializacion = NodoAsignacion(None, nombre, expresion)
            
        # Condición
        condicion = self.expresion()
        self.coincidir('DELIMITER')  # ";"
        
        # Incremento
        incremento = self.incremento()
        self.coincidir('DELIMITER')  # ")"
        
        # Cuerpo
        cuerpo = []
        if self.obtener_token_actual() and self.obtener_token_actual()[1] == "{":
            self.coincidir('DELIMITER')  # "{"
            cuerpo = self.cuerpo()  # Cuerpo del for
            self.coincidir('DELIMITER')  # "}"
        else:
            cuerpo.append(self.instruccion_unica())
            
        return NodoFor(inicializacion, condicion, incremento, cuerpo)

    def condicional_if(self):
        self.coincidir('KEYWORD')  # if
        self.coincidir('DELIMITER')  # "("
        condicion = self.expresion()  # Condición del if
        self.coincidir('DELIMITER')  # ")"

        cuerpo = []
        if self.obtener_token_actual() and self.obtener_token_actual()[1] == "{":
            self.coincidir('DELIMITER')  # "{"
            cuerpo = self.cuerpo()  # Cuerpo del if
            self.coincidir('DELIMITER')  # "}"
        else:
            cuerpo.append(self.instruccion_unica())
            
        # Verificar si hay else
        cuerpo_else = []
        if self.obtener_token_actual() and self.obtener_token_actual()[1] == "else":
            self.coincidir('KEYWORD')  # else
            if self.obtener_token_actual() and self.obtener_token_actual()[1] == "{":
                self.coincidir('DELIMITER')  # "{"
                cuerpo_else = self.cuerpo()  # Cuerpo del else
                self.coincidir('DELIMITER')  # "}"
            else:
                cuerpo_else.append(self.instruccion_unica())
            
        return NodoIf(condicion, cuerpo, cuerpo_else)

    def condicional_else(self):
        self.coincidir('KEYWORD')  # else
        
        cuerpo = []
        if self.obtener_token_actual() and self.obtener_token_actual()[1] == "{":
            self.coincidir('DELIMITER')  # "{"
            cuerpo = self.cuerpo()  # Cuerpo del else
            self.coincidir('DELIMITER')  # "}"
        else:
            cuerpo.append(self.instruccion_unica())
            
        return NodoElse(cuerpo)

    def imprimir(self):
        self.coincidir('KEYWORD')  # print
        self.coincidir('DELIMITER')  # "("
        
        token_actual = self.obtener_token_actual()
        if token_actual[0] == 'STRING':
            token = self.coincidir('STRING')
            expresion = NodoString(token[1])
        else:
            expresion = self.expresion()  # Expresion a imprimir
    
        self.coincidir('DELIMITER')  # ")"
        self.coincidir('DELIMITER')  # ";"
        
        return NodoPrint(expresion)

In [ ]:
class NodoAST:
    # Clase base para todos los nodos del AST
    def to_dict(self):
        # Método para convertir nodo a diccionario (para JSON)
        raise NotImplementedError("Debe implementarse en las subclases")
    
    def optimizacion(self):
        return self
    
    def traducir(self):
        raise NotImplementedError("Metodo traducir() no implementado en este nodo")

In [ ]:
class NodoPrograma(NodoAST):
    def __init__(self, funciones):
        self.funciones = funciones
        
    def to_dict(self):
        return {
            "tipo": "programa",
            "funciones": [funcion.to_dict() for funcion in self.funciones]
        }
    
    def tiene_main(self):
        for funcion in self.funciones:
            if funcion.nombre == "main":
                return True
        return False

In [ ]:
class NodoFuncion(NodoAST):
    def __init__(self, tipo, nombre, parametros, cuerpo):
        self.tipo = tipo
        self.nombre = nombre
        self.parametros = parametros
        self.cuerpo = cuerpo
    
    def traducir(self):
        params = ".".join(p.traducir() for p in self.parametros)
        cuerpo = "\n    ".join(c.traducir() for c in self.cuerpo)
        return f"def {self.nombre}({params}):\n    {cuerpo}"
        

    def to_dict(self):
        return {
            "tipo": "funcion",
            "retorno": self.tipo,
            "nombre": self.nombre,
            "parametros": [param.to_dict() for param in self.parametros],
            "cuerpo": [instr.to_dict() for instr in self.cuerpo]
        }

In [ ]:
class NodoParametro(NodoAST):
    def __init__(self, tipo, nombre):
        self.tipo = tipo
        self.nombre = nombre

    def traducir(self):
        return self.nombre
    
    def to_dict(self):
        return {
            "tipo": "parametro",
            "tipo_dato": self.tipo,
            "nombre": self.nombre
        }

In [ ]:
class NodoAsignacion(NodoAST):
    def __init__(self, tipo, nombre, expresion):
        self.tipo = tipo
        self.nombre = nombre
        self.expresion = expresion
        
    def traducir(self):
        return f"{self.nombre} = {self.expresion.traducir()}"
        
    def to_dict(self):
        return {
            "tipo": "asignacion",
            "tipo_dato": self.tipo,
            "nombre": self.nombre,
            "expresion": self.expresion.to_dict()
        }

In [ ]:
class NodoOperacion(NodoAST):
    def __init__(self, izquierda, operador, derecha):
        self.izquierda = izquierda
        self.operador = operador
        self.derecha = derecha
             
    def to_dict(self):
        return {
            "tipo": "operacion",
            "operador": self.operador,
            "izquierda": self.izquierda.to_dict(),
            "derecha": self.derecha.to_dict()
        }
        
    def traducir(self):
        if self.operador == '+':
            return f"({self.izquierda.traducir()} + {self.derecha.traducir()})"
        elif self.operador == '-':
            return f"({self.izquierda.traducir()} - {self.derecha.traducir()})"
        elif self.operador == '*':
            return f"({self.izquierda.traducir()} * {self.derecha.traducir()})"
        elif self.operador == '/':
            return f"({self.izquierda.traducir()} / {self.derecha.traducir()})"
        
    def optimizacion(self):
        izquierda = self.izquierda.optimizacion()
        derecha = self.derecha.optimizacion()
        
        # Si ambos operandos son numeros, evaluamos la operacion
        if isinstance(izquierda, self.NodoNumero) and isinstance(derecha, self.NodoNumero):
            if self.operador == '+':
                return self.NodoNumero(float(izquierda.valor) + float(derecha.valor))
            elif self.operador == '-':
                return self.NodoNumero(float(izquierda.valor) - float(derecha.valor))
            elif self.operador == '*':
                return self.NodoNumero(float(izquierda.valor) * float(derecha.valor))  # Corregido: * en lugar de +
            elif self.operador == '/' and float(derecha.valor) != 0:
                return self.NodoNumero(float(izquierda.valor) / float(derecha.valor))
            
        # Simplificacion algebraica
        if self.operador == '*' and isinstance(derecha, self.NodoNumero) and float(derecha.valor) == 1:
            return izquierda
        if self.operador == '*' and isinstance(izquierda, self.NodoNumero) and float(izquierda.valor) == 1:
            return derecha
        if self.operador == '*' and isinstance(derecha, self.NodoNumero) and float(derecha.valor) == 0:
            return derecha
        if self.operador == '*' and isinstance(izquierda, self.NodoNumero) and float(izquierda.valor) == 0:
            return izquierda
        
        # Si no se puede optimizar mas, devolvemos la misma operacion
        return NodoOperacion(izquierda, self.operador, derecha)

In [ ]:
class NodoRetorno(NodoAST):
    def __init__(self, expresion):
        self.expresion = expresion
    
    def traducir(self):
        return f"return {self.expresion.traducir()}"
    def to_dict(self):
        return {
            "tipo": "retorno",
            "expresion": self.expresion.to_dict()
        }

In [ ]:
class NodoIdentificador(NodoAST):
    def __init__(self, nombre):
        self.nombre = nombre
        
    def to_dict(self):
        return {
            "tipo": "identificador",
            "nombre": self.nombre
        }
    
    def optimizacion(self):
        return self
    
    def traducir(self):
        return self.nombre

In [ ]:
class NodoNumero(NodoAST):
    def __init__(self, valor):
        self.valor = valor
        
    def to_dict(self):
        return {
            "tipo": "numero",
            "valor": self.valor
        }
    
    def optimizacion(self):
        return self
    
    def traducir(self):
        return str(self.valor)

In [ ]:
class NodoString(NodoAST):
    def __init__(self, valor):
        self.valor = valor
        
    def to_dict(self):
        return {
            "tipo": "string",
            "valor": self.valor
        }

In [ ]:
class NodoIf(NodoAST):
    def __init__(self, condicion, cuerpo, cuerpo_else=None):
        self.condicion = condicion
        self.cuerpo = cuerpo
        self.cuerpo_else = cuerpo_else if cuerpo_else else []
        
    def to_dict(self):
        return {
            "tipo": "if",
            "condicion": self.condicion.to_dict(),
            "cuerpo": [instr.to_dict() for instr in self.cuerpo],
            "else": [instr.to_dict() for instr in self.cuerpo_else]
        }
        
    def traducir(self):
        cuerpo_if = "\n    ".join(c.traducir() for c in self.cuerpo)
        if self.cuerpo_else:
            cuerpo_else = "\n    ".join(c.traducir() for c in self.cuerpo_else)
            return f"if {self.condicion.traducir()}:\n    {cuerpo_if}\nelse:\n    {cuerpo_else}"
        return f"if {self.condicion.traducir()}:\n    {cuerpo_if}"
        
    def optimizacion(self):
        # Optimizar la condición
        condicion_opt = self.condicion.optimizacion()
        
        # Optimizar los cuerpos
        cuerpo_opt = [inst.optimizacion() for inst in self.cuerpo if inst is not None]
        cuerpo_else_opt = [inst.optimizacion() for inst in self.cuerpo_else if inst is not None]
        
        if isinstance(condicion_opt, NodoNumero):
            if float(condicion_opt.valor) != 0:  # Condición siempre verdadera
                return cuerpo_opt
            else:  # Condición siempre falsa
                return cuerpo_else_opt
                
        return NodoIf(condicion_opt, cuerpo_opt, cuerpo_else_opt)

In [ ]:
class NodoWhile(NodoAST):
    def __init__(self, condicion, cuerpo):
        self.condicion = condicion
        self.cuerpo = cuerpo
        
    def to_dict(self):
        return {
            "tipo": "while",
            "condicion": self.condicion.to_dict(),
            "cuerpo": [instr.to_dict() for instr in self.cuerpo]
        }

In [ ]:
class NodoFor(NodoAST):
    def __init__(self, inicializacion, condicion, incremento, cuerpo):
        self.inicializacion = inicializacion
        self.condicion = condicion
        self.incremento = incremento
        self.cuerpo = cuerpo
        
    def to_dict(self):
        return {
            "tipo": "for",
            "inicializacion": self.inicializacion.to_dict(),
            "condicion": self.condicion.to_dict(),
            "incremento": self.incremento.to_dict(),
            "cuerpo": [instr.to_dict() for instr in self.cuerpo]
        }

In [ ]:
class NodoIncremento(NodoAST):
    def __init__(self, nombre, operador):
        self.nombre = nombre
        self.operador = operador
        
    def to_dict(self):
        return {
            "tipo": "incremento",
            "nombre": self.nombre,
            "operador": self.operador
        }

In [ ]:
class NodoPrint(NodoAST):
    def __init__(self, expresion):
        self.expresion = expresion
        
    def to_dict(self):
        return {
            "tipo": "print",
            "expresion": self.expresion.to_dict()
        }

In [ ]:
class NodoElse(NodoAST):
    def __init__(self, cuerpo):
        self.cuerpo = cuerpo
        
    def to_dict(self):
        return {
            "tipo": "else",
            "cuerpo": [instr.to_dict() for instr in self.cuerpo]
        }

In [ ]:
class NodoLlamadaFuncion(NodoAST):
    def __init__(self, nombre, argumentos):
        self.nombre = nombre
        self.argumentos = argumentos
        
    def to_dict(self):
        return {
            "tipo": "llamada_funcion",
            "nombre": self.nombre,
            "argumentos": [arg.to_dict() for arg in self.argumentos]
        }

In [ ]:
def ast_a_json(ast):
    return json.dumps(ast.to_dict(), indent=2)

In [ ]:
# Función para imprimir el AST de forma legible
def imprimir_ast(nodo, nivel=0):
    prefijo = "  " * nivel
    
    if isinstance(nodo, NodoPrograma):
        print(f"{prefijo}Programa:")
        for funcion in nodo.funciones:
            imprimir_ast(funcion, nivel + 1)
        print(f"{prefijo}¿Tiene función main?: {'Sí' if nodo.tiene_main() else 'No'}")
        
    elif isinstance(nodo, NodoFuncion):
        print(f"{prefijo}Función: {nodo.nombre} (Tipo de retorno: {nodo.tipo})")
        print(f"{prefijo}Parámetros:")
        for param in nodo.parametros:
            imprimir_ast(param, nivel + 1)
        print(f"{prefijo}Cuerpo:")
        for instr in nodo.cuerpo:
            imprimir_ast(instr, nivel + 1)
            
    elif isinstance(nodo, NodoParametro):
        print(f"{prefijo}Parámetro: {nodo.nombre} (Tipo: {nodo.tipo})")
        
    elif isinstance(nodo, NodoAsignacion):
        tipo_str = f"(Tipo: {nodo.tipo})" if nodo.tipo else ""
        print(f"{prefijo}Asignación: {nodo.nombre} {tipo_str}")
        print(f"{prefijo}  Valor:")
        imprimir_ast(nodo.expresion, nivel + 2)
        
    elif isinstance(nodo, NodoOperacion):
        print(f"{prefijo}Operación: {nodo.operador}")
        print(f"{prefijo}  Izquierda:")
        imprimir_ast(nodo.izquierda, nivel + 2)
        print(f"{prefijo}  Derecha:")
        imprimir_ast(nodo.derecha, nivel + 2)
        
    elif isinstance(nodo, NodoRetorno):
        print(f"{prefijo}Retorno:")
        imprimir_ast(nodo.expresion, nivel + 1)
        
    elif isinstance(nodo, NodoIdentificador):
        print(f"{prefijo}Identificador: {nodo.nombre}")
        
    elif isinstance(nodo, NodoNumero):
        print(f"{prefijo}Número: {nodo.valor}")
        
    elif isinstance(nodo, NodoString):
        print(f"{prefijo}String: {nodo.valor}")
        
    elif isinstance(nodo, NodoIf):
        print(f"{prefijo}If:")
        print(f"{prefijo}  Condición:")
        imprimir_ast(nodo.condicion, nivel + 2)
        print(f"{prefijo}  Cuerpo:")
        for instr in nodo.cuerpo:
            imprimir_ast(instr, nivel + 2)
            
    elif isinstance(nodo, NodoWhile):
        print(f"{prefijo}While:")
        print(f"{prefijo}  Condición:")
        imprimir_ast(nodo.condicion, nivel + 2)
        print(f"{prefijo}  Cuerpo:")
        for instr in nodo.cuerpo:
            imprimir_ast(instr, nivel + 2)
            
    elif isinstance(nodo, NodoFor):
        print(f"{prefijo}For:")
        print(f"{prefijo}  Inicialización:")
        imprimir_ast(nodo.inicializacion, nivel + 2)
        print(f"{prefijo}  Condición:")
        imprimir_ast(nodo.condicion, nivel + 2)
        print(f"{prefijo}  Incremento:")
        imprimir_ast(nodo.incremento, nivel + 2)
        print(f"{prefijo}  Cuerpo:")
        for instr in nodo.cuerpo:
            imprimir_ast(instr, nivel + 2)
            
    elif isinstance(nodo, NodoIncremento):
        print(f"{prefijo}Incremento: {nodo.nombre} {nodo.operador}")
        
    elif isinstance(nodo, NodoPrint):
        print(f"{prefijo}Print:")
        imprimir_ast(nodo.expresion, nivel + 1)
        
    elif isinstance(nodo, NodoElse):
        print(f"{prefijo}Else:")
        print(f"{prefijo}  Cuerpo:")
        for instr in nodo.cuerpo:
            imprimir_ast(instr, nivel + 2)
            
    elif isinstance(nodo, NodoLlamadaFuncion):
        print(f"{prefijo}Llamada a función: {nodo.nombre}")
        print(f"{prefijo}  Argumentos:")
        for arg in nodo.argumentos:
            imprimir_ast(arg, nivel + 2)
            
    else:
        print(f"{prefijo}Nodo desconocido: {type(nodo)}")

In [ ]:
# Ejemplo de uso con múltiples funciones
texto_prueba = """
int suma(int a, int b) {
    return a + b;
}

int multiplica(int a, int b) {
    return a * b;
}

int main() {
    int resultado = 0;
    int x = 5;
    int y = 10;
    
    resultado = suma(x, y);
    print(resultado);
    
    resultado = multiplica(x, y);
    print(resultado);
    
    return 0;
}
"""

In [ ]:
# Análisis léxico
tokens_ejercicio = identificar(texto_prueba)

In [ ]:
try:
    # Análisis sintáctico y construcción del AST
    parser = Parser(tokens_ejercicio)
    ast_raiz = parser.parsear()
    
    print('Análisis sintáctico exitoso')
    
    # Convertir el AST a JSON y mostrarlo
    print("\n===== AST en formato JSON =====")
    json_ast = ast_a_json(ast_raiz)
    print(json_ast)
    
except SyntaxError as e:
    print(e)

In [ ]:
codigo_fuente = '''
    int suma(int a, int b) {
        int c = a + 0;
        return c;
    }'''

In [ ]:
tokens = identificar(codigo_fuente)

parser_codigofuente = Parser(tokens)
arbol_ast = parser_codigofuente.parsear()

arbol_ast_optimizado = arbol_ast.optimizacion()

codigo_traducido = ""
for funcion in arbol_ast_optimizado.funciones:
    codigo_traducido += funcion.traducir() + "\n\n"

print("Código traducido y optimizado:")
print(codigo_traducido)

In [ ]:
class GeneradorEnsamblador:
    def __init__(self):
        self.codigo = []
        self.etiqueta_contador = 0
        self.tabla_simbolos = {}
        self.registros = ['eax', 'ebx', 'ecx', 'edx']
        self.registro_actual = 0
        self.espacio_local = 0
        self.etiquetas_loop = []
        
    def nueva_etiqueta(self, prefijo="L"):
        self.etiqueta_contador += 1
        return f"{prefijo}_{self.etiqueta_contador}"
        
    def generar(self, nodo):
        if isinstance(nodo, NodoPrograma):
            self.codigo.append("section .text")
            self.codigo.append("global _start")
            self.codigo.append("_start:")
            self.codigo.append("  call main")
            self.codigo.append("  mov ebx, eax") 
            self.codigo.append("  mov eax, 1")    # sys_exit
            self.codigo.append("  int 0x80")
            self.codigo.append("")
            
            for funcion in nodo.funciones:
                self.generar(funcion)
                
        elif isinstance(nodo, NodoFuncion):
            # Reiniciar estado para cada función
            self.tabla_simbolos = {}
            self.espacio_local = 0
            
            # Prologo de la funcion
            self.codigo.append(f"{nodo.nombre}:")
            self.codigo.append("  push ebp")
            self.codigo.append("  mov ebp, esp")
            
            # Calcular espacio necesario para variables locales
            for instruccion in nodo.cuerpo:
                if isinstance(instruccion, NodoAsignacion) and instruccion.tipo:
                    self.tabla_simbolos[instruccion.nombre] = self.espacio_local + 4
                    self.espacio_local += 4
                    
            if self.espacio_local > 0:
                self.codigo.append(f"  sub esp, {self.espacio_local}")
                
            # Generar parámetros
            for i, param in enumerate(nodo.parametros):
                self.tabla_simbolos[param.nombre] = 8 + i * 4  # ebp+8 es el primer parámetro
                
            # Generar cuerpo de la función
            for instruccion in nodo.cuerpo:
                self.generar(instruccion)
                
            # Agregar return
            if not any(isinstance(i, NodoRetorno) for i in nodo.cuerpo):
                if nodo.tipo != "void":
                    self.codigo.append("  mov eax, 0")  # Valor de retorno por defecto
                self.codigo.append("  mov esp, ebp")
                self.codigo.append("  pop ebp")
                self.codigo.append("  ret")
                
        elif isinstance(nodo, NodoIf):
            # Condicion
            self.generar(nodo.condicion)
            
            # etiquetas
            etiqueta_else = self.nueva_etiqueta("else")
            etiqueta_fin = self.nueva_etiqueta("endif")
            
            # saltar condicion
            self.codigo.append("  cmp eax, 0")
            self.codigo.append(f"  je {etiqueta_else}")
            
            # if
            for instruccion in nodo.cuerpo:
                self.generar(instruccion)
            
            # Saltar al final para evitar el else
            self.codigo.append(f"  jmp {etiqueta_fin}")
            
            # Código del else
            self.codigo.append(f"{etiqueta_else}:")
            for instruccion in nodo.cuerpo_else:
                self.generar(instruccion)
            
            # Etiqueta de fin
            self.codigo.append(f"{etiqueta_fin}:")
            
        elif isinstance(nodo, NodoWhile):
            etiqueta_inicio = self.nueva_etiqueta("while_start")
            etiqueta_fin = self.nueva_etiqueta("while_end")
            
            # Guarda etiquetas para break/continue
            self.etiquetas_loop.append((etiqueta_fin, etiqueta_inicio))
            
            self.codigo.append(f"{etiqueta_inicio}:")
            
            # Generar condición
            self.generar(nodo.condicion)
            self.codigo.append("  cmp eax, 0")
            self.codigo.append(f"  je {etiqueta_fin}")
            
            # Cuerpo del while
            for instruccion in nodo.cuerpo:
                self.generar(instruccion)
                
            self.codigo.append(f"  jmp {etiqueta_inicio}")
            self.codigo.append(f"{etiqueta_fin}:")
            
            # Sacar etiquetas del stack
            self.etiquetas_loop.pop()
            
        elif isinstance(nodo, NodoFor):
            etiqueta_inicio = self.nueva_etiqueta("for_start")
            etiqueta_fin = self.nueva_etiqueta("for_end")
            etiqueta_incremento = self.nueva_etiqueta("for_inc")
            
            # Inicialización
            self.generar(nodo.inicializacion)
            
            # Guardar etiquetas para break/continue
            self.etiquetas_loop.append((etiqueta_fin, etiqueta_incremento))
            
            self.codigo.append(f"{etiqueta_inicio}:")
            
            # Condición
            self.generar(nodo.condicion)
            self.codigo.append("  cmp eax, 0")
            self.codigo.append(f"  je {etiqueta_fin}")
            
            # Cuerpo
            for instruccion in nodo.cuerpo:
                self.generar(instruccion)
                
            # Incremento
            self.codigo.append(f"{etiqueta_incremento}:")
            self.generar(nodo.incremento)
            
            self.codigo.append(f"  jmp {etiqueta_inicio}")
            self.codigo.append(f"{etiqueta_fin}:")
            
            # Sacar etiquetas del stack
            self.etiquetas_loop.pop()
            
        elif isinstance(nodo, NodoAsignacion):
            # Generar la expresión
            self.generar(nodo.expresion)
            
            # Guardar el resultado
            if nodo.nombre in self.tabla_simbolos:
                offset = self.tabla_simbolos[nodo.nombre]
                self.codigo.append(f"  mov [ebp - {offset}], eax")
            else:
                # Es una variable local nueva
                self.espacio_local += 4
                offset = self.espacio_local
                self.tabla_simbolos[nodo.nombre] = offset
                self.codigo.append(f"  mov [ebp - {offset}], eax")
                
        elif isinstance(nodo, NodoOperacion):
            self.generar(nodo.izquierda)
            self.codigo.append("  push eax")
            self.generar(nodo.derecha)
            self.codigo.append("  pop ebx")
            
            if nodo.operador == '+':
                self.codigo.append("  add eax, ebx")
            elif nodo.operador == '-':
                self.codigo.append("  sub eax, ebx")
            elif nodo.operador == '*':
                self.codigo.append("  imul eax, ebx")
            elif nodo.operador == '/':
                self.codigo.append("  cdq")
                self.codigo.append("  idiv ebx")
            elif nodo.operador == '==':
                self.codigo.append("  cmp ebx, eax")
                self.codigo.append("  sete al")
                self.codigo.append("  movzx eax, al")
            elif nodo.operador == '!=':
                self.codigo.append("  cmp ebx, eax")
                self.codigo.append("  setne al")
                self.codigo.append("  movzx eax, al")
            elif nodo.operador == '<':
                self.codigo.append("  cmp ebx, eax")
                self.codigo.append("  setl al")
                self.codigo.append("  movzx eax, al")
            elif nodo.operador == '>':
                self.codigo.append("  cmp ebx, eax")
                self.codigo.append("  setg al")
                self.codigo.append("  movzx eax, al")
            elif nodo.operador == '<=':
                self.codigo.append("  cmp ebx, eax")
                self.codigo.append("  setle al")
                self.codigo.append("  movzx eax, al")
            elif nodo.operador == '>=':
                self.codigo.append("  cmp ebx, eax")
                self.codigo.append("  setge al")
                self.codigo.append("  movzx eax, al")
            elif nodo.operador == '&&':
                self.codigo.append("  and eax, ebx")
                self.codigo.append("  cmp eax, 0")
                self.codigo.append("  setne al")
                self.codigo.append("  movzx eax, al")
            elif nodo.operador == '||':
                self.codigo.append("  or eax, ebx")
                self.codigo.append("  cmp eax, 0")
                self.codigo.append("  setne al")
                self.codigo.append("  movzx eax, al")
                
        elif isinstance(nodo, NodoRetorno):
            if nodo.expresion:
                self.generar(nodo.expresion)
            else:
                self.codigo.append("  mov eax, 0")  # Valor de retorno por defecto
                
            self.codigo.append("  mov esp, ebp")
            self.codigo.append("  pop ebp")
            self.codigo.append("  ret")
            
        elif isinstance(nodo, NodoNumero):
            self.codigo.append(f"  mov eax, {nodo.valor}")
            
        elif isinstance(nodo, NodoIdentificador):
            offset = self.tabla_simbolos[nodo.nombre]
            if offset > 0:  # Variable local
                self.codigo.append(f"  mov eax, [ebp - {offset}]")
            else:  # Parámetro 
                self.codigo.append(f"  mov eax, [ebp + {8 + offset}]")
                
        elif isinstance(nodo, NodoLlamadaFuncion):
            # Empujar argumentos en la pila (en orden inverso)
            for arg in reversed(nodo.argumentos):
                self.generar(arg)
                self.codigo.append("  push eax")
                
            self.codigo.append(f"  call {nodo.nombre}")
            
            # Limpiar la pila de argumentos
            if nodo.argumentos:
                self.codigo.append(f"  add esp, {len(nodo.argumentos) * 4}")
                
        elif isinstance(nodo, NodoPrint):
            # Generar código para imprimir números de cualquier tamaño
            self.codigo.append("  ; Inicio de print")
            self.generar(nodo.expresion)
            
            # Agregar función de conversión de número a string
            if not hasattr(self, 'print_func_added'):
                self.print_func_added = True
                self.codigo.extend([
                    "",
                    "print_number:",
                    "  ; Función para imprimir un número en eax",
                    "  pusha",
                    "  mov ecx, 10",
                    "  mov ebx, buffer + 11",
                    "  mov byte [ebx], 0",
                    "",
                    "print_loop:",
                    "  dec ebx",
                    "  xor edx, edx",
                    "  div ecx",
                    "  add dl, '0'",
                    "  mov [ebx], dl",
                    "  test eax, eax",
                    "  jnz print_loop",
                    "",
                    "  mov eax, 4",
                    "  mov ecx, ebx",
                    "  mov edx, buffer + 12",
                    "  sub edx, ecx",
                    "  mov ebx, 1",
                    "  int 0x80",
                    "",
                    "  popa",
                    "  ret",
                    ""
                ])
            
            self.codigo.extend([
                "  call print_number",
                "  ; Fin de print"
            ])
            
            # Buffer definido
            if not hasattr(self, 'bss_section'):
                self.bss_section = []
            if "buffer resb 12" not in self.bss_section:
                self.bss_section.append("buffer resb 12")
            
        elif isinstance(nodo, NodoIncremento):
            offset = self.tabla_simbolos[nodo.nombre]
            self.codigo.append(f"  mov eax, [ebp - {offset}]")
            if nodo.operador == '++':
                self.codigo.append("  inc eax")
            else:
                self.codigo.append("  dec eax")
            self.codigo.append(f"  mov [ebp - {offset}], eax")
            
        elif isinstance(nodo, NodoString):
            # Almacenar string en la sección .data
            if not hasattr(self, 'data_section'):
                self.data_section = []
                self.string_labels = {}
                
            if nodo.valor not in self.string_labels:
                label = self.nueva_etiqueta("str")
                self.string_labels[nodo.valor] = label
                self.data_section.append(f'{label} db "{nodo.valor}", 0')
                
            self.codigo.append(f"  mov eax, {self.string_labels[nodo.valor]}")
            
        else:
            raise ValueError(f"Nodo no soportado: {type(nodo)}")
            
    def obtener_codigo(self):
        # Secciones
        sections = []
        
        # Sección .data si hay strings
        if hasattr(self, 'data_section'):
            sections.append("section .data")
            sections.extend(self.data_section)
        
        # Sección .bss si hay buffers
        if hasattr(self, 'bss_section'):
            sections.append("section .bss")
            sections.extend(self.bss_section)
        
        # Sección .text (siempre presente)
        sections.append("section .text")
        sections.append("global _start")
        sections.extend(self.codigo)
        
        # Filtrar líneas vacías innecesarias
        filtered = []
        for line in sections:
            if line.strip() or (not filtered or filtered[-1].strip()):
                filtered.append(line)
        
        return "\n".join(filtered)

In [ ]:
class Optimizador:
    def optimizar(self, nodo):
        if isinstance(nodo, NodoPrograma):
            nodo.funciones = [self.optimizar(funcion) for funcion in nodo.funciones]
            return nodo
            
        elif isinstance(nodo, NodoFuncion):
            nuevo_cuerpo = []
            for instr in nodo.cuerpo:
                instr_opt = self.optimizar(instr)
                if instr_opt is not None:  # Puede ser None si se eliminó
                    if isinstance(instr_opt, list):  # Algunas optimizaciones devuelven listas
                        nuevo_cuerpo.extend(instr_opt)
                    else:
                        nuevo_cuerpo.append(instr_opt)
            nodo.cuerpo = nuevo_cuerpo
            return nodo
            
        elif isinstance(nodo, NodoOperacion):
            # Optimizar subexpresiones
            nodo.izquierda = self.optimizar(nodo.izquierda)
            nodo.derecha = self.optimizar(nodo.derecha)
            
            # Plegado de constantes
            if isinstance(nodo.izquierda, NodoNumero) and isinstance(nodo.derecha, NodoNumero):
                try:
                    if nodo.operador == '+':
                        return NodoNumero(float(nodo.izquierda.valor) + float(nodo.derecha.valor))
                    elif nodo.operador == '-':
                        return NodoNumero(float(nodo.izquierda.valor) - float(nodo.derecha.valor))
                    elif nodo.operador == '*':
                        return NodoNumero(float(nodo.izquierda.valor) * float(nodo.derecha.valor))
                    elif nodo.operador == '/' and float(nodo.derecha.valor) != 0:
                        return NodoNumero(float(nodo.izquierda.valor) / float(nodo.derecha.valor))
                    elif nodo.operador == '==':
                        return NodoNumero(1 if float(nodo.izquierda.valor) == float(nodo.derecha.valor) else 0)
                    elif nodo.operador == '!=':
                        return NodoNumero(1 if float(nodo.izquierda.valor) != float(nodo.derecha.valor) else 0)
                    elif nodo.operador == '<':
                        return NodoNumero(1 if float(nodo.izquierda.valor) < float(nodo.derecha.valor) else 0)
                    elif nodo.operador == '>':
                        return NodoNumero(1 if float(nodo.izquierda.valor) > float(nodo.derecha.valor) else 0)
                    elif nodo.operador == '<=':
                        return NodoNumero(1 if float(nodo.izquierda.valor) <= float(nodo.derecha.valor) else 0)
                    elif nodo.operador == '>=':
                        return NodoNumero(1 if float(nodo.izquierda.valor) >= float(nodo.derecha.valor) else 0)
                except:
                    pass  # Si hay error en la conversión, mantener la operación
                    
            # Simplificaciones algebraicas
            if nodo.operador == '+' and isinstance(nodo.derecha, NodoNumero) and float(nodo.derecha.valor) == 0:
                return nodo.izquierda
            if nodo.operador == '*' and isinstance(nodo.derecha, NodoNumero) and float(nodo.derecha.valor) == 1:
                return nodo.izquierda
            if nodo.operador == '*' and isinstance(nodo.derecha, NodoNumero) and float(nodo.derecha.valor) == 0:
                return NodoNumero(0)
            if nodo.operador == '&&' and isinstance(nodo.izquierda, NodoNumero) and float(nodo.izquierda.valor) == 0:
                return NodoNumero(0)
            if nodo.operador == '||' and isinstance(nodo.izquierda, NodoNumero) and float(nodo.izquierda.valor) != 0:
                return NodoNumero(1)
                
            return nodo
            
        elif isinstance(nodo, NodoIf):
            nodo.condicion = self.optimizar(nodo.condicion)
            
            # Optimizar cuerpos
            nuevo_cuerpo = []
            for instr in nodo.cuerpo:
                opt_instr = self.optimizar(instr)
                if opt_instr is not None:
                    nuevo_cuerpo.append(opt_instr)
            nodo.cuerpo = nuevo_cuerpo
            
            nuevo_cuerpo_else = []
            for instr in nodo.cuerpo_else:
                opt_instr = self.optimizar(instr)
                if opt_instr is not None:
                    nuevo_cuerpo_else.append(opt_instr)
            nodo.cuerpo_else = nuevo_cuerpo_else
            
            # Si la condición es constante
            if isinstance(nodo.condicion, NodoNumero):
                if float(nodo.condicion.valor) != 0:
                    return nodo.cuerpo  # Devolver solo el cuerpo del if
                else:
                    return nodo.cuerpo_else  # Devolver solo el cuerpo del else
                    
            # Si el cuerpo del if está vacío y no hay else
            if not nodo.cuerpo and not nodo.cuerpo_else:
                return None
                
            return nodo
            
        elif isinstance(nodo, NodoWhile):
            nodo.condicion = self.optimizar(nodo.condicion)
            
            # Optimizar cuerpo
            nuevo_cuerpo = []
            for instr in nodo.cuerpo:
                opt_instr = self.optimizar(instr)
                if opt_instr is not None:
                    nuevo_cuerpo.append(opt_instr)
            nodo.cuerpo = nuevo_cuerpo
            
            # Si la condición es falsa constante, eliminar el while
            if isinstance(nodo.condicion, NodoNumero) and float(nodo.condicion.valor) == 0:
                return None
                
            return nodo
            
        elif isinstance(nodo, NodoFor):
            nodo.inicializacion = self.optimizar(nodo.inicializacion)
            nodo.condicion = self.optimizar(nodo.condicion)
            nodo.incremento = self.optimizar(nodo.incremento)
            
            # Optimizar cuerpo
            nuevo_cuerpo = []
            for instr in nodo.cuerpo:
                opt_instr = self.optimizar(instr)
                if opt_instr is not None:
                    nuevo_cuerpo.append(opt_instr)
            nodo.cuerpo = nuevo_cuerpo
            
            
        elif isinstance(nodo, NodoAsignacion):
            nodo.expresion = self.optimizar(nodo.expresion)
            
            # Eliminar asignaciones redundantes (x = x)
            if isinstance(nodo.expresion, NodoIdentificador) and nodo.nombre == nodo.expresion.nombre:
                return None
                
            return nodo
            
        elif isinstance(nodo, NodoPrint):
            # Asegurarnos de que existe la sección bss
            if not hasattr(self, 'bss_section'):
                self.bss_section = []
            
            # Agregar buffer si no existe
            if "buffer resb 12" not in self.bss_section:
                self.bss_section.append("buffer resb 12")
                    
            nodo.expresion = self.optimizar(nodo.expresion)
            return nodo
            
        elif isinstance(nodo, (NodoRetorno, NodoIdentificador, NodoNumero, NodoString, 
                                NodoLlamadaFuncion, NodoIncremento, NodoElse)):
            # Nodos que no necesitan optimización especial
            return nodo
            
        else:
            raise ValueError(f"Tipo de nodo no reconocido: {type(nodo)}")

In [ ]:
class GeneradorBinario:
    def __init__(self):
        self.ensamblador = GeneradorEnsamblador()
        
    def compilar(self, ast, archivo_salida="salida"):
        # 1. Optimizar el AST
        optimizador = Optimizador()
        ast_optimizado = optimizador.optimizar(ast)
        
        # 2. Generar código ensamblador
        self.ensamblador.generar(ast_optimizado)
        codigo_asm = self.ensamblador.obtener_codigo()
        
        # 3. Escribir archivo .asm
        with open(f"{archivo_salida}.asm", "w") as f:
            f.write(codigo_asm)
            print(f"Archivo {archivo_salida}.asm generado correctamente")
            
        try:
            print("Ensamblando con NASM...")
            subprocess.run(["nasm", "-f", "win32", f"{archivo_salida}.asm", "-o", f"{archivo_salida}.obj"], check=True)
            print("Ensamblado completado")
            
            print("Enlazando con ld...")
            subprocess.run(["ld", "-m", "i386pe", "-s", "-o", f"{archivo_salida}.exe", f"{archivo_salida}.obj"], check=True)
            print("Enlazado completado")
            
            print(f"Compilación exitosa. Ejecutable generado: {archivo_salida}.exe")
            return True
        except subprocess.CalledProcessError as e:
            print(f"Error durante la compilación (código {e.returncode}): {e}")
            if e.stdout:
                print("Salida estándar:", e.stdout.decode())
            if e.stderr:
                print("Error estándar:", e.stderr.decode())
            return False
        except FileNotFoundError:
            print("Error: NASM o ld no están instalados o no están en el PATH")
            print("Asegúrate de que:")
            print("1. NASM esté instalado y en el PATH")
            print("2. MinGW o similar esté instalado para proporcionar ld")
            return False
    def ejecutar_en_nueva_ventana(archivo_exe):
        try:
            subprocess.run(f"start cmd /k {archivo_exe}", shell=True, check=True)
            return True
        except subprocess.CalledProcessError as e:
            print(f"Error al ejecutar: {e}")
            return False

In [ ]:
# Código fuente de ejemplo
codigo_fuente = """
int factorial(int n) {
    if (n <= 1) {
        return 1;
    } else {
        return n * factorial(n - 1);
    }
}

int main() {
    int num = 2;
    int resultado = factorial(num);
    print(resultado);
    return 0;
}
"""

# 1. Análisis léxico
tokens = identificar(codigo_fuente)

# 2. Análisis sintáctico
parser = Parser(tokens)
ast = parser.parsear()

# 3. Optimización
optimizador = Optimizador()
ast_optimizado = optimizador.optimizar(ast)

# 4. Generación de código
generador_bin = GeneradorBinario()
exito = generador_bin.compilar(ast_optimizado, "factorial")

# 5. Mostrar código ensamblador generado
if exito:
    print("\nCódigo ensamblador generado:")
    print(generador_bin.ensamblador.obtener_codigo())
    
# Uso en cmd
GeneradorBinario.ejecutar_en_nueva_ventana("factorial.exe")